In [100]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
 #Read in the csv from WeatherPy
city = pd.read_csv("../WeatherPy/output_data/cities.csv")
city.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.29,-156.79,10.40,78,90,10.29,US,1604970601
1,1,vaini,-21.20,-175.20,82.40,78,40,10.29,TO,1604970679
2,2,upata,8.01,-62.40,80.01,87,100,1.01,VE,1604970740
3,3,port elizabeth,-33.92,25.57,55.40,82,0,11.41,ZA,1604970699
4,4,langxiang,46.95,128.88,19.42,84,98,5.08,CN,1604971089


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [102]:
 #use gmap, configure with api key
gmaps.configure(api_key=g_key)

In [103]:
#Create a heatmap
fig = gmaps.figure(center=(46, -5.0), zoom_level=1.5)
fig.add_layer(gmaps.heatmap_layer(city[["Lat", "Lng"]], weights=city["Humidity"], dissipating=False, max_intensity=300, point_radius=3))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
 #My perfect vacation city will have a temperature of between 71 and 83 F, and wind speed of below 5 mph and no cloudiness at all.
new_df = city.loc[(city["Max Temp"] < 83) & (city["Wind Speed"] < 5) & (city["Max Temp"] > 71) & (city["Cloudiness"] == 0)].dropna()
new_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,riyadh,24.69,46.72,71.60,49,0,4.70,SA,1604970661
74,74,sur,22.57,59.53,74.05,73,0,0.94,OM,1604970687
142,142,salalah,17.02,54.09,75.20,60,0,3.36,OM,1604970694
228,228,doha,25.29,51.53,77.00,83,0,3.36,QA,1604971112
259,259,januaria,-15.48,-44.37,72.90,65,0,4.50,BR,1604971116
277,277,flinders,-34.58,150.86,73.99,58,0,4.00,AU,1604971118
440,440,malakal,9.53,31.66,73.87,32,0,4.27,SS,1604971137


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
 #hotels df has city, country, latitude and longitute
hotels = new_df[["City", "Country", "Lat", "Lng"]].copy()
hotels["Hotel Name"] = ""
hotels

,City,Country,Lat,Lng,Hotel Name
11,riyadh,SA,24.69,46.72,
74,sur,OM,22.57,59.53,
142,salalah,OM,17.02,54.09,
228,doha,QA,25.29,51.53,
259,januaria,BR,-15.48,-44.37,
277,flinders,AU,-34.58,150.86,
440,malakal,SS,9.53,31.66,


In [106]:
#create parameters for google maps api
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# for each of the hotels in dataframe interate through and make requests 
for i, r in hotels.iterrows():
    lat = r["Lat"]
    lng = r["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # make request
    names = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params)
    names = names.json()
    
    #save hotel name to hotels dataframe
    try:
        hotels.loc[i, "Hotel Name"] = names["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing info... skipping.")

hotels

,City,Country,Lat,Lng,Hotel Name
11,riyadh,SA,24.69,46.72,InterContinental Riyadh
74,sur,OM,22.57,59.53,Sur Plaza Hotel
142,salalah,OM,17.02,54.09,HAMDAN PLAZA HOTEL
228,doha,QA,25.29,51.53,Doha Marriott Hotel
259,januaria,BR,-15.48,-44.37,Hotel Paraguassu
277,flinders,AU,-34.58,150.86,Shellharbour Resort & Conference Centre
440,malakal,SS,9.53,31.66,Nile Palace Hotel


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
locations = hotels[["Lat", "Lng"]]

In [108]:
#On top of the heat map, put down pins for each of the ideal hotels with hotel info
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))